In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
tamil_train_data = pd.read_csv("/content/drive/My Drive/AAAA--DATASETS/tamil_train.tsv",sep='\t')
tamil_val_data = pd.read_csv("/content/drive/My Drive/AAAA--DATASETS/tamil_dev.tsv",sep='\t')
tamil_test_data = pd.read_csv("/content/drive/My Drive/AAAA--DATASETS/tamil_test_answer.tsv",sep='\t')

In [ ]:
import re
tamil_train_data['text'] = tamil_train_data['text'].apply(lambda x: x.lower())
# removing special chars
tamil_train_data['text'] = tamil_train_data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
# removing consecutive letters
tamil_train_data['text'] = tamil_train_data['text'].apply((lambda x: re.sub(r'(.)\1+', r'\1',x)))

tamil_train_data['category'] = tamil_train_data['category'].apply((lambda x: re.sub(r'\s+', '', x, flags=re.I)))

tamil_train_data.head()

,text,category
0,trailer late ah parthavanga like podunga,Positive
1,move pathutu vanthu trailer pakurvnga yaru,Positive
2,puthupetai dhanush ah yarelam pathinga,Positive
3,dhanush oda character puthu sa erukay mas ta,Positive
4,vera level ipa pesungada moka nu thalaiva,Positive


In [ ]:
tamil_val_data['text'] = tamil_val_data['text'].apply(lambda x: x.lower())
# removing special chars
tamil_val_data['text'] = tamil_val_data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
# removing consecutive letters
tamil_val_data['text'] = tamil_val_data['text'].apply((lambda x: re.sub(r'(.)\1+', r'\1',x)))

tamil_val_data['category'] = tamil_val_data['category'].apply((lambda x: re.sub(r'\s+', '', x, flags=re.I)))

tamil_val_data.tail()

,text,category
1255,sapypathy vijayama bigilu umpi get more disli...,Negative
1256,stylea erukana ha ha naturaly,Positive
1257,style la irukana hahaha mas dialogue,Negative
1258,like for lokesh kanagaraj thalapathy 64 director,unknown_state
1259,na indha trailer 30 time pathuten paka paka t...,Positive


In [ ]:
tamil_test_data['text'] = tamil_test_data['text'].apply(lambda x: x.lower())
# removing special chars
tamil_test_data['text'] = tamil_test_data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
# removing consecutive letters
tamil_test_data['text'] = tamil_test_data['text'].apply((lambda x: re.sub(r'(.)\1+', r'\1',x)))

tamil_test_data['category'] = tamil_test_data['category'].apply((lambda x: re.sub(r'\s+', '', x, flags=re.I)))

tamil_test_data.tail()

,id,text,category
3144,ta_sent_3145,tamil krish ah irukum oh,Positive
3145,ta_sent_3146,thalaiva trailer ye pataiya kelapudhe sekiram ...,Positive
3146,ta_sent_3147,inum neraya neraya neraya neraya neraya,Mixed_feelings
3147,ta_sent_3148,105 to 130 vere level mas,Positive
3148,ta_sent_3149,pangalingala13k like 40m view varamum inum ner...,Positive


In [ ]:
data = pd.concat([tamil_train_data, tamil_val_data], ignore_index=True)
data = pd.concat([data, tamil_test_data], ignore_index=True)
print(data)

                                                    text  ...            id
0               trailer late ah parthavanga like podunga  ...           NaN
1             move pathutu vanthu trailer pakurvnga yaru  ...           NaN
2                 puthupetai dhanush ah yarelam pathinga  ...           NaN
3           dhanush oda character puthu sa erukay mas ta  ...           NaN
4              vera level ipa pesungada moka nu thalaiva  ...           NaN
...                                                  ...  ...           ...
15739                           tamil krish ah irukum oh  ...  ta_sent_3145
15740  thalaiva trailer ye pataiya kelapudhe sekiram ...  ...  ta_sent_3146
15741            inum neraya neraya neraya neraya neraya  ...  ta_sent_3147
15742                          105 to 130 vere level mas  ...  ta_sent_3148
15743  pangalingala13k like 40m view varamum inum ner...  ...  ta_sent_3149

[15744 rows x 3 columns]


In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

max_features = 1500
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X= tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)
print(X[:2])
print(len(X))

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    5
   894   15 1383    2   57]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
  1384  157  311    5  133]]
15744


In [ ]:
Y = pd.get_dummies(data['category']).values
print(Y)
len(Y)

[[0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 ...
 [1 0 0 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]]


15744

In [ ]:
train = len(tamil_train_data)
val = len(tamil_val_data)

In [ ]:
X_train = X[:train]
Y_train = Y[:train]

print(len(X_train))
print(len(Y_train))

X_val = X[train : (train+val)]
Y_val = Y[train : (train+val)]

print(len(X_val))
print(len(Y_val))

X_test = X[(train+val) : ]
Y_test = Y[(train+val) : ]

print(len(X_test))
print(len(Y_test))

11335
11335
1260
1260
3149
3149


In [ ]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, Bidirectional , Dropout , LSTM

In [ ]:
model = Sequential()
model.add(Embedding(max_features, 256))
model.add(Bidirectional(LSTM(128)))
#model.add(Dense(128))    #for LSTM 4
model.add(Dense(5,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='rmsprop',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 256)         384000    
_________________________________________________________________
bidirectional (Bidirectional (None, 256)               394240    
_________________________________________________________________
dense (Dense)                (None, 5)                 1285      
Total params: 779,525
Trainable params: 779,525
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
#model.fit(X_train, Y_train, epochs = 1, batch_size= 32 ,verbose = 1 , validation_data=(X_val,Y_val))

In [ ]:
#model.save("/content/drive/My Drive/AAAA--DATASETS/LSTM 3.h5")
model.load_weights('/content/drive/My Drive/AAAA--DATASETS/LSTM 3.h5')

In [ ]:
Y_pred = model.predict(X_test)

In [ ]:
Y_pred[:5]

array([[1.26719847e-01, 1.13768235e-01, 6.03618503e-01, 5.12327962e-02,
        1.04660586e-01],
       [2.42158696e-01, 9.84933153e-02, 6.03960812e-01, 3.42536671e-03,
        5.19618727e-02],
       [1.40645087e-01, 4.79608215e-02, 8.06764305e-01, 7.02041143e-04,
        3.92771559e-03],
       [1.97477013e-01, 3.26942235e-01, 3.56217623e-01, 3.82146575e-02,
        8.11484084e-02],
       [1.97759405e-01, 1.03907488e-01, 6.21415138e-01, 1.38907805e-02,
        6.30272254e-02]], dtype=float32)

In [ ]:
classes = ['Mixed_feelings','Negative','Positive','not-Tamil','unknown_state']

In [ ]:
import numpy as np
pred_class = []
for item in Y_pred:
  index = np.argmax(item)
  pred_class.append(classes[index])
  #pred_class.append(index)

print(pred_class)

['Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Mixed_feelings', 'Positive', 'not-Tamil', 'unknown_state', 'Positive', 'Positive', 'Positive', 'unknown_state', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Mixed_feelings', 'Positive', 'Negative', 'Positive', 'Positive', 'not-Tamil', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Negative', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Negative', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'not-Tamil', 'Positive', 'Positive', 'Positive', 'Negative', 'n

In [ ]:
len(pred_class)

3149

In [ ]:
print("\n Training Accuracy :" , model.evaluate(X_train, Y_train  , verbose = 2)[1] ,"\n")
print("\n Testing Accuracy :" , model.evaluate(X_test, Y_test , verbose = 2)[1] )

355/355 - 7s - loss: 0.8011 - accuracy: 0.7186

 Training Accuracy : 0.7185707688331604 

99/99 - 2s - loss: 0.9076 - accuracy: 0.6818

 Testing Accuracy : 0.6818037629127502


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(pred_class,tamil_test_data['category']))

                precision    recall  f1-score   support

Mixed_feelings       0.02      0.29      0.03        24
      Negative       0.24      0.46      0.31       218
      Positive       0.95      0.71      0.81      2796
     not-Tamil       0.46      0.70      0.55        66
 unknown_state       0.12      0.44      0.18        45

      accuracy                           0.68      3149
     macro avg       0.36      0.52      0.38      3149
  weighted avg       0.87      0.68      0.76      3149



SUBMISSION PREPARATION

In [ ]:
print(pred_class.count('Mixed_feelings'))
print(pred_class.count('Negative'))
print(pred_class.count('Positive'))
print(pred_class.count('not-Tamil'))
print(pred_class.count('unknown_state'))

In [ ]:
rnn_data = pd.read_csv("/content/drive/My Drive/AAAA--DATASETS/tamil_test.tsv",sep='\t')
print(rnn_data)
rnn_data['label'] = pred_class

compression_opts = dict(method='zip',archive_name='HRS-TECHIE_Tamil.tsv')  
rnn_data.to_csv('HRS-TECHIE.zip', sep='\t', index=False, compression=compression_opts)
print(rnn_data)